In [1]:
import pandas as pd
from os import listdir
import re
import psycopg2
import secret


### Grab all the data from the data folder and load it into a dataframe

In [2]:
files = listdir('data')

data_all = []

for i in files:

    file_path = "data/{}".format(i)
    
    df = pd.read_csv(file_path, on_bad_lines = 'skip')
    df['Make'] = i[:-4]
    df = df.drop(columns=["Unnamed: 0"])
    
    data_all.append(df)
    
df_all = pd.concat(data_all, axis = 0, ignore_index=True)

In [3]:
df_all.head()
df_transform = df_all.copy()

In [4]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125140 entries, 0 to 125139
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Name          125140 non-null  object 
 1   Mileage       125139 non-null  object 
 2   Dealer Name   125140 non-null  object 
 3   Rating        125114 non-null  float64
 4   Rating Count  125128 non-null  object 
 5   Price         125140 non-null  object 
 6   Make          125140 non-null  object 
dtypes: float64(1), object(6)
memory usage: 6.7+ MB


# Transformation

In [5]:
df_transform = df_transform.dropna()
df_transform.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 125113 entries, 0 to 125139
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Name          125113 non-null  object 
 1   Mileage       125113 non-null  object 
 2   Dealer Name   125113 non-null  object 
 3   Rating        125113 non-null  float64
 4   Rating Count  125113 non-null  object 
 5   Price         125113 non-null  object 
 6   Make          125113 non-null  object 
dtypes: float64(1), object(6)
memory usage: 7.6+ MB


In [6]:
def capitalize(make):
    
    make = re.sub("_", " ", make)
    make = make.split(" ")
    
    make = [i.capitalize() for i in make]
    
    make = " ".join(make)
    
    return make
    
df_transform['Make'] = df_all['Make'].apply(lambda x: capitalize(x))
df_transform['Make'].value_counts()

Ford             10100
Toyota           10099
Jeep             10098
Nissan           10098
Chevrolet        10097
Honda            10085
Mercedes Benz     9586
Bmw               7461
Subaru            6324
Kia               6188
Cadillac          5218
Gmc               4614
Ram               4569
Volkswagen        3809
Acura             3590
Volvo             3411
Porsche           2598
Infiniti          2542
Buick             1993
Chrysler          1073
Mini               603
Maserati           313
Aston Martin       219
Alfa Romeo         173
Rolls Royce        138
Fiat                66
Mitsubishi          48
Name: Make, dtype: int64

### Extracting the year

In [7]:
def extract_date(name):
    return int(re.search(r"(\d{4})", name).group(1))

df_transform['Name'].apply(lambda x: extract_date(x))

0         2020
1         2021
2         2020
3         2020
4         2019
          ... 
125135    2019
125136    2019
125137    2020
125138    2018
125139    2019
Name: Name, Length: 125113, dtype: int64

In [8]:
df_transform['Year'] = df_all['Name'].apply(lambda x: extract_date(x))

df_transform.head(10)

,Name,Mileage,Dealer Name,Rating,Rating Count,Price,Make,Year
0,2020 Acura MDX 3.5L w/Technology Package,"36,762 mi.",Spreen Acura,4.7,(996 reviews),"$42,870",Acura,2020
1,2021 Acura RDX A-Spec,"13,919 mi.",Advantage Acura of Naperville,4.7,(432 reviews),"$46,900",Acura,2021
2,2020 Acura MDX 3.5L,"21,979 mi.",Vandergriff Acura,4.7,"(1,355 reviews)","$40,876",Acura,2020
3,2020 Acura MDX 3.5L,"23,967 mi.",Pohanka Acura,4.8,"(1,661 reviews)","$40,221",Acura,2020
4,2019 Acura RDX Base,"26,249 mi.",Davis Acura,4.7,"(2,599 reviews)","$34,338",Acura,2019
5,2021 Acura TLX A-Spec,"16,679 mi.",Vandergriff Acura,4.7,"(1,355 reviews)","$43,998",Acura,2021
6,2019 Acura MDX 3.5L w/Technology Package,"22,247 mi.",Buerkle Acura,4.9,"(1,004 reviews)","$43,895",Acura,2019
7,2020 Acura TLX FWD,"24,810 mi.",Davis Acura,4.7,"(2,599 reviews)","$32,388",Acura,2020
8,2021 Acura RDX Technology Package,"4,791 mi.",Paragon Acura,4.6,"(2,734 reviews)","$42,516",Acura,2021
9,2021 Acura RDX Base,"15,517 mi.",Phil Smith Acura,4.8,"(1,291 reviews)","$37,160",Acura,2021


In [9]:
def clean_mileage(miles):
    
    mileage = re.sub(',', '', miles)
    mileage = re.sub(' mi.', '', mileage)
    
    return int(mileage)

df_transform['Mileage'] = df_transform['Mileage'].apply(lambda x: clean_mileage(x))

df_transform.head(10)

,Name,Mileage,Dealer Name,Rating,Rating Count,Price,Make,Year
0,2020 Acura MDX 3.5L w/Technology Package,36762,Spreen Acura,4.7,(996 reviews),"$42,870",Acura,2020
1,2021 Acura RDX A-Spec,13919,Advantage Acura of Naperville,4.7,(432 reviews),"$46,900",Acura,2021
2,2020 Acura MDX 3.5L,21979,Vandergriff Acura,4.7,"(1,355 reviews)","$40,876",Acura,2020
3,2020 Acura MDX 3.5L,23967,Pohanka Acura,4.8,"(1,661 reviews)","$40,221",Acura,2020
4,2019 Acura RDX Base,26249,Davis Acura,4.7,"(2,599 reviews)","$34,338",Acura,2019
5,2021 Acura TLX A-Spec,16679,Vandergriff Acura,4.7,"(1,355 reviews)","$43,998",Acura,2021
6,2019 Acura MDX 3.5L w/Technology Package,22247,Buerkle Acura,4.9,"(1,004 reviews)","$43,895",Acura,2019
7,2020 Acura TLX FWD,24810,Davis Acura,4.7,"(2,599 reviews)","$32,388",Acura,2020
8,2021 Acura RDX Technology Package,4791,Paragon Acura,4.6,"(2,734 reviews)","$42,516",Acura,2021
9,2021 Acura RDX Base,15517,Phil Smith Acura,4.8,"(1,291 reviews)","$37,160",Acura,2021


In [10]:
def clean_rating_count(rating_count):
    
    words_to_remove = ['\(', '\)', ' reviews', ',', ' review']
    
    for i in words_to_remove:
        rating_count = re.sub(i, '', rating_count)
    
    return int(rating_count)

df_transform['Rating Count'] = df_transform['Rating Count'].apply(lambda x: clean_rating_count(x))

In [11]:
def clean_price(price):
    
    price = re.sub(',', '', price)
    price = price.strip('$')
    
    if price == 'Not Priced':
        return None
    
    return int(price)

df_transform['Price'] = df_transform['Price'].apply(lambda x: clean_price(x))

### Final Transormation

In [12]:
df_transform.head(15)

,Name,Mileage,Dealer Name,Rating,Rating Count,Price,Make,Year
0,2020 Acura MDX 3.5L w/Technology Package,36762,Spreen Acura,4.7,996,42870.0,Acura,2020
1,2021 Acura RDX A-Spec,13919,Advantage Acura of Naperville,4.7,432,46900.0,Acura,2021
2,2020 Acura MDX 3.5L,21979,Vandergriff Acura,4.7,1355,40876.0,Acura,2020
3,2020 Acura MDX 3.5L,23967,Pohanka Acura,4.8,1661,40221.0,Acura,2020
4,2019 Acura RDX Base,26249,Davis Acura,4.7,2599,34338.0,Acura,2019
5,2021 Acura TLX A-Spec,16679,Vandergriff Acura,4.7,1355,43998.0,Acura,2021
6,2019 Acura MDX 3.5L w/Technology Package,22247,Buerkle Acura,4.9,1004,43895.0,Acura,2019
7,2020 Acura TLX FWD,24810,Davis Acura,4.7,2599,32388.0,Acura,2020
8,2021 Acura RDX Technology Package,4791,Paragon Acura,4.6,2734,42516.0,Acura,2021
9,2021 Acura RDX Base,15517,Phil Smith Acura,4.8,1291,37160.0,Acura,2021


### Final Product

In [13]:
df_transform = df_transform.dropna()
df_transform.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 122483 entries, 0 to 125138
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Name          122483 non-null  object 
 1   Mileage       122483 non-null  int64  
 2   Dealer Name   122483 non-null  object 
 3   Rating        122483 non-null  float64
 4   Rating Count  122483 non-null  int64  
 5   Price         122483 non-null  float64
 6   Make          122483 non-null  object 
 7   Year          122483 non-null  int64  
dtypes: float64(2), int64(3), object(3)
memory usage: 8.4+ MB


# Load - Load the data into the database

### Connect to the database

In [14]:
conn = psycopg2.connect(
    host = secret.ENDPOINT,
    user = secret.USERNAME,
    password = secret.PASSWORD,
    database = secret.DATABASE,
    port = secret.PORT
)

cur = conn.cursor()

### Create the table

In [25]:
cur.execute("DROP TABLE IF EXISTS cars")
conn.commit()

In [26]:
cur.execute(
    """CREATE TABLE cars(
	ID SERIAL PRIMARY KEY,
	model VARCHAR(100),
	make VARCHAR(20),
	mileage int8,
	dealer_name VARCHAR(200),
	rating float8,
	rating_cnt int8,
	price float8,
	year int8
)"""
)

conn.commit()

### Test script to see if it works

### Iterating through the dataframe and inserting every value (Will take a while to complete)

In [28]:
insert_script = "INSERT INTO cars (model, make, dealer_name, mileage, rating, rating_cnt, price, year) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"

batch_insert = []

for i, row in df_transform.iterrows():
    
    insert_value = (str(row['Name']), str(row['Make']), str(row['Dealer Name']), int(row['Mileage']), float(row['Rating']), int(row['Rating Count']), float(row['Price']), int(row['Year']))
    batch_insert.append(insert_value)
    
    new_str = insert_script.format(insert_value)

    
# insert into the database
cur.executemany(
    insert_script, batch_insert
)

conn.commit()


In [29]:
cur.execute("SELECT * FROM cars")

len(cur.fetchall())

122483

### Closing the connection

In [30]:
if cur is not None:
    cur.close()
if conn is not None:
    conn.close()